In [1]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


The below script needs to be able to find the current output cell; this is an easy method to get it.

 This tutorial demostrates the **definition**, resp. the **import** of 
 data for **boundary and initial values**. 
 
 In order to demonstrate the usage, 
 we employ the exemplaric **Poisson solver**.

In [2]:
using BoSSS.Application.SipPoisson;

We use a **temporary database** for this tutorial:

In [3]:
var tempDb = CreateTempDatabase();

Creating database 'C:\Users\flori\AppData\Local\Temp\553852612'.


We use the following helper function to create a **template for the multiple solver runs**.

In [4]:
Func<SipControl> PreDefinedControl = delegate() {
    SipControl c = new SipControl();
 
    c.SetDGdegree(2);
 
    c.GridFunc = delegate() {
        // define a grid of 10x10 cells
        double[] nodes = GenericBlas.Linspace(-1, 1, 11);
        var grd = Grid2D.Cartesian2DGrid(nodes, nodes);
 
        // set the entire boundary to Dirichlet b.c.
        grd.DefineEdgeTags(delegate (double[] X) {
            return BoundaryType.Dirichlet.ToString();
        });
 
        return grd;
    };
 
    c.SetDatabase(tempDb);
    c.savetodb = true; 
 
    return c;    
};

Again, we are using the **workflow management**

In [5]:
BoSSSshell.WorkflowMgm.Init("Demo_BoundaryAndInitialData");

Project name is set to 'Demo_BoundaryAndInitialData'.


# **1. Textual and Embedded formulas**


In [6]:
SipControl c1 = PreDefinedControl();

Provide **initial data** as a text:

In [7]:
c1.AddInitialValue("RHS","X => Math.Sin(X[0])*Math.Cos(X[1])",
                        TimeDependent:false);

Finally, all initial data is stored in the 
***AppControl.InitialVa*** dictionary and 
all boundary data is stored in the 
***AppControl.BoundaryValues*** dictionary.
 
The common interface for all varinats to specify boundary
and initial data is ***IBoundaryAndInitialData***.

The snippet above is only a shortcut to add a ***Formula*** object,
which implements the ***IBoundaryAndInitialData*** interface.

In [8]:
c1.InitialValues

key,
RHS,BoSSS.Solution.Control.Formula


In [9]:
c1.InitialValues["RHS"]

BoSSS.Solution.Control.Formula

In **BoSSSpad**, such objects can also be extracted from 
static methods of classes; note that these should not depend on any other
object in the worksheet.

In [10]:
Formula BndyFormula = new Formula(
    "BndyValue.BndyFunction",
    false,
"static class BndyValue {"+
"    public static double BndyFunction(double[] X) {"+
"        return 1.0;"+
"    }"+
"}");

In [11]:
c1.AddBoundaryValue(BoundaryType.Dirichlet.ToString(),
                    "T",
                    BndyFormula);

Creates a Job named **J1** and runs it

In [12]:
var J1 = c1.RunBatch();

Control object contains grid function. Trying to Serialize the grid...
Grid Edge Tags changed.
Control object modified.
Deploying job UnnamedJob_1 ... 
Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\Demo_BoundaryAndInitialData-ipPoisson2021Sep05_085808
copied 42 files.
   written file: control.obj
deployment finished.



The next line prints the Status of the Job **J1**.

In [13]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

Timeout.


We can print the Status of the Job **J1**.

In [14]:
J1.Status

FinishedSuccessful

We can also check via a method if the Job **J1** is truly finished

In [15]:

NUnit.Framework.Assert.IsTrue(J1.Status == JobStatus.FinishedSuccessful);

# **2. 1D Splines**

**Splines** can be used to interpolate nodal data onto a DG field;
currently, only 1D is supported.

In [16]:
SipControl c2 = PreDefinedControl();

In [17]:
// create test data for the spline
double[] xNodes = GenericBlas.Linspace(-2,2,13);
double[] yNodes = xNodes.Select(x => x*0.4).ToArray();

In [18]:
var rhsSpline = new Spline1D(xNodes, yNodes,
                             0,
                             Spline1D.OutOfBoundsBehave.Extrapolate);

In [19]:
/// BoSSScmdSilent
double err = 0;
// test the spline: a line must be interpolated exactly.
foreach(double xtst in GenericBlas.Linspace(-3,3,77)) { 
   double sVal = rhsSpline.Evaluate(new double[] {xtst , 0, 0 }, 0.0);
   double rVal = xtst*0.4;
   err += Math.Abs(sVal - rVal);
}
NUnit.Framework.Assert.Less(err, 1.0e-10, "Slpine implementation fail.");

In [20]:
c2.AddInitialValue("RHS", rhsSpline);

In [21]:
var J2 = c2.RunBatch();

Control object contains grid function. Trying to Serialize the grid...
Grid Edge Tags changed.
An equivalent grid (b22511f5-53a7-43d4-bb2c-470feb1a1989) is already present in the database -- the grid will not be saved.
Control object modified.
Deploying job UnnamedJob_2 ... 
Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\Demo_BoundaryAndInitialData-ipPoisson2021Sep05_085819
copied 42 files.
   written file: control.obj
deployment finished.



In [22]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

Timeout.


In [23]:
J2.Status

FinishedSuccessful

In [24]:
/// BoSSScmdSilent
NUnit.Framework.Assert.IsTrue(J2.Status == JobStatus.FinishedSuccessful);


# **3. Interpolating values from other Calculations**

For demonstrational purposes, we use the result (i.e. the last time-step) 
of a previous calculation as a right-hand-side for the next calculation.

In [25]:
var j2Sess = J2.LatestSession;

In [26]:
j2Sess

Demo_BoundaryAndInitialData	UnnamedJob_2	09/05/2021 08:58:22	e0c45358...

In [27]:
j2Sess.Timesteps

#0:  { Time-step: 0; Physical time: 0s; Fields: T, ResidualKP1, Error, MgLevel_0; Name:  }
#1:  { Time-step: 1; Physical time: 0s; Fields: T, ResidualKP1, Error, MgLevel_0; Name:  }


In [28]:
var lastTimeStep = j2Sess.Timesteps.Last();

We encapsulate the value **T** in the **ForeignGridValue** object,
which allows interpolation between different meshes:

In [29]:
var newForeignMesh = new ForeignGridValue(lastTimeStep,"T");

In [30]:
/// Use different mesh in the control file:
SipControl c3 = PreDefinedControl();

In [31]:
c3.GridFunc = delegate() {
   // define a grid of *triangle* cells
   double[] nodes = GenericBlas.Linspace(-1, 1, 11);
   var grd = Grid2D.UnstructuredTriangleGrid(nodes, nodes);
 
   // set the entire boundary to Dirichlet b.c.
   grd.DefineEdgeTags(delegate (double[] X) {
       return BoundaryType.Dirichlet.ToString();
   });
 
   return grd;
};
// we also save the RHS in the database
c3.AddFieldOption("RHS", SaveOpt: FieldOpts.SaveToDBOpt.TRUE);

In [32]:
/// finally, we define the RHS:
c3.AddInitialValue("RHS", newForeignMesh);

In [33]:
/// BoSSScmdSilent
double orgProbe = newForeignMesh.Evaluate(new double[] {0.5,0.5}, 0.0);
double newProbe = lastTimeStep.GetField("T").ProbeAt(new double[] {0.5,0.5});
NUnit.Framework.Assert.Less(Math.Abs(orgProbe - newProbe), 1.0e-10, "Check (1) on ForeignGridValue failed");

In [34]:
var J3 = c3.RunBatch();

Control object contains grid function. Trying to Serialize the grid...
Grid Edge Tags changed.
Control object modified.
Deploying job UnnamedJob_3 ... 
Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\Demo_BoundaryAndInitialData-ipPoisson2021Sep05_085832
copied 42 files.
   written file: control.obj
deployment finished.



In [35]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

Timeout.


In [36]:
J3.Status

InProgress

In [37]:
/// BoSSScmdSilent
NUnit.Framework.Assert.IsTrue(J3.Status == JobStatus.FinishedSuccessful);

Since the quadrilateral mesh used for the original
right-hand-side is geometrically embedded in the triangular mesh 
the **interpolation error should be zero** (up to machine precision).

In [38]:
var firstTimeStep = J3.LatestSession.Timesteps.First();

In [39]:
DGField RhsOnTriangles = firstTimeStep.GetField("rhs"); // case-insensitive!
DGField RhsOriginal    = lastTimeStep.GetField("T");

In [40]:
// note: we have to cast DGField to ConventionalDGField in order to use
// the 'L2Distance' function:
((ConventionalDGField)RhsOnTriangles).L2Distance((ConventionalDGField)RhsOriginal)

1.200592953981901E-17

In [41]:
/// BoSSScmdSilent
var H1err = ((ConventionalDGField)RhsOnTriangles).H1Distance((ConventionalDGField)RhsOriginal);
NUnit.Framework.Assert.Less(H1err, 1.0e-10, "Check (2) on ForeignGridValue failed.");

# **4. Restart from Dummy-Sessions**

Dummy sessions are kind of fake siolver runs, with the only purpose 
of using them for a restart.

In [42]:
DGField RHSforRestart = firstTimeStep.GetField("RHS");

We save the DG field ***RHSforRestart*** in the database;
This automatically creates a timestep and a session which host the DG field:

In [43]:
var RestartTimestep = tempDb.SaveTimestep(RHSforRestart);

In [44]:
RestartTimestep

 { Time-step: 0; Physical time: 0s; Fields: RHS; Name:  }

In [45]:
RestartTimestep.Session

Demo_BoundaryAndInitialData	InitialValueSession	09/05/2021 08:58:44	df886820...

This time step can be used as a restart value.:

In [46]:
var c4 = PreDefinedControl();

In [47]:
c4.InitialValues.Clear();
c4.SetRestart(RestartTimestep);

In [48]:
var J4 = c4.RunBatch();

Deploying job UnnamedJob_4 ... 
Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\Demo_BoundaryAndInitialData-ipPoisson2021Sep05_085845
copied 42 files.
   written file: control.obj
deployment finished.



In [49]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

All jobs finished.


In [50]:
J4.Status

FinishedSuccessful

In [51]:
/// BoSSScmdSilent
NUnit.Framework.Assert.IsTrue(J4.Status == JobStatus.FinishedSuccessful);

# Note

Since no mesh interpolation is performed for the restart, it is much faster
than **ForeignGridValue**, but less flexible 
(a restart is always performed on the same mesh).

To avoid multiple mesh interpolations (e.g. when multiple runs are required)
one coudl therefore speed up the process by doing the 
mesh interpolation once ***use ProjectFromForeignGrid*** in BoSSSpad and
save the interpolation in a dummy session.